# Exercise13

### s1300102
### Aku Haga

## Data


In [17]:
import pandas as pd
from PAMI.extras.DF2DB import denseDF2DB as pro

# Step 1: Read the air_pollution_data CSV file into a dataframe
dataset = pd.read_csv('air_pollution_data.csv')

# Step 2: Remove the "Timestamp" column
dataset.drop(columns=['Timestamp'], inplace=True, axis=1)

# Step 3: Replace NaN values with zero
dataset = dataset.fillna(0)

# Step 4: Replace values greater than or equal to 100 with zero
dataset.where(dataset <= 100, 0, inplace=True)

# Step 5: Convert the dataframe into a transactional database with items whose value is greater than or equal to 15
db = pro.denseDF2DB(inputDF=dataset, thresholdValue=15, condition='>=')

# Convert and store the dataframe as a transactional database file
db.createTransactional(outputFile='PM24HeavyPollutionRecordingSensors.csv')

# Step 6: Read the generated CSV file
csv_file_path = 'PM24HeavyPollutionRecordingSensors.csv'
df = pd.read_csv(csv_file_path, sep=',')

# Convert the dataframe to a list
df_list = df.values.tolist()

# Create a new list and split each element using tab as the separator
new_list = [row[0].split('\t') for row in df_list]

# Convert the list to a new dataframe
new_df = pd.DataFrame(new_list)

# Save the new dataframe as a CSV file
new_df.to_csv(csv_file_path, index=False)


   1101010  1101020  1101520  1101540  1102010  1102020  1102510  1103010  \
0      0.0      0.0      4.0      3.0      2.0     11.0     -2.0      0.0   
1      0.0      0.0      3.0      3.0      2.0     11.0      4.0      0.0   
2      0.0      0.0      3.0      3.0      2.0      9.0      4.0      0.0   
3      0.0      0.0      4.0      6.0      2.0      0.0      8.0      0.0   
4      0.0      0.0      4.0      4.0      0.0      4.0     -1.0      0.0   

   1103520  1104030  ...  46208010  46212010  46220010  46225010  46302010  \
0      1.0      2.0  ...       7.0       5.0       7.0       3.0       0.0   
1      0.0      2.0  ...       8.0       6.0       5.0       5.0       0.0   
2      6.0      1.0  ...       6.0       7.0       7.0       3.0       0.0   
3      6.0      3.0  ...       8.0       8.0       7.0       3.0       0.0   
4      3.0      5.0  ...       7.0       6.0       7.0       4.0       0.0   

   46302020  46321010  46466010  46482010  47301950  
0       0.0   

In [15]:
!pip install -U PAMI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.5 MB/s eta 0:00:00
  Created wheel for JsonForm: filename=JsonForm-0.0.2-py3-none-any.whl size=3311 sha256=cc8115e247dc7d83f412c240cb6b0ce9f040a3151f53b3fe6c05b9c47a6293db
  Stored in directory: /root/.cache/pip/wheels/b6/e5/87/11026246d3bd4ad67c0615682d2d6748bbd9a40ac0490882bd
  Created wheel for JsonSir: filename=JsonSir-0.0.2-py3-none-any.whl size=4752 sha256=100eed1d48609828947cc503d18d3fe0c05898e1607c4f0e10f91a6a0df71078
  Stored in directory: /root/.cache/pi

## Knowledge discovery


In [ ]:
from PAMI.frequentPattern.basic import FPGrowth as alg
inputFile = 'PM24HeavyPollutionRecordingSensors.csv'

#specify the constraints used in the model
minSup = 1
#create the object of the mining algorithm
obj = alg.FPGrowth(inputFile, minSup)
#start the mining process
obj.startMine()
#Save the generated patterns in a file
obj.save('frequentPatterns.txt')

## Visualization

In [ ]:
import re
import pandas as pd
import plotly.express as px

# Step 1: Read the file to get patterns
with open('frequentPatterns.txt', 'r') as file:
    patterns = file.readlines()

# Step 2: Analyze the patterns to identify the longest pattern
max_pattern = []  # Initialize a list to store the longest pattern
max_length = 0    # Initialize the length of the longest pattern

for pattern in patterns:
    # Obtain coordinates in the pattern using regular expression
    coordinates = re.findall(r'Point\((.*?)\)', pattern)
    if len(coordinates) > max_length:
        max_pattern = coordinates  # Update the longest pattern
        max_length = len(coordinates)  # Update the length of the longest pattern

# Step 3: Display the longest pattern on Plotly Express Open Street Map
# Convert the list of coordinates to a DataFrame
df = pd.DataFrame(max_pattern, columns=['coordinate'])

# Split the 'coordinate' column into 'Lon' and 'Lat' columns
df[['Lon', 'Lat']] = df['coordinate'].str.split(' ', expand=True)

# Convert 'Lon' and 'Lat' columns to float type
df['Lon'] = df['Lon'].astype(float)
df['Lat'] = df['Lat'].astype(float)

# Create a scatter plot map using Plotly Express
fig = px.scatter_mapbox(
    df,
    lat='Lat',
    lon='Lon',
    hover_name='coordinate'
)

# Update the map style to 'open-street-map'
fig.update_layout(mapbox_style='open-street-map')

# Display the map
fig.show()
